# Radar Plan Position Indicator (PPI) in 3D

This is adopted from [this example](https://docs.openradarscience.org/projects/xradar/en/stable/notebooks/plot-ppi.html) to visualize sweeps of radar data as a 3D volume. The 3D visualization here help us gather insight into the 3D structure storms.

In [ ]:
import xarray as xr
import xradar as xd
import pyvista as pv
import pvxarray  # Import to register accessor
from open_radar_data import DATASETS

pv.set_plot_theme("document")
pv.set_jupyter_backend("server")

Fetching CfRadial1 radar data file from open-radar-data repository.

In [ ]:
filename = DATASETS.fetch("cfrad.20080604_002217_000_SPOL_v36_SUR.nc")
radar = xd.io.open_cfradial1_datatree(filename)
radar = radar.xradar.georeference()  # Add georeferencing to plot in 3D
list(radar.children)

Now we need to stack all of the sweeps on top of each other to create a 3D data volume

In [ ]:
ds_list = []
for key in list(radar.children):
    if "sweep" in key:
        ds_list.append(radar[key].ds.drop_duplicates(dim="azimuth"))
ds = xr.concat(ds_list, dim="sweep")
ds

Use `pyvista-xarray`'s accessor to create a PyVista mesh from the data volume. This will build a curvilinear grid (`pyvista.StructuredGrid`)

In [ ]:
mesh = ds["DBZ"].pyvista.mesh(x="x", y="y", z="z")
mesh

Preview the data volume

In [ ]:
mesh.plot(clim=(-20, 60))

Pseudo-volume rendering with transparent contours

In [ ]:
pl = pv.Plotter()
pl.add_mesh(mesh.contour(), opacity=0.9, clim=(-20, 60), lighting=False)
pl.show(jupyter_backend="static")